In [5]:
import sys
sys.path.append('../velorama')
import gaston_dynamics as gd
import scanpy as sc
import numpy as np 
import pandas as pd

In [6]:
import os
from collections import defaultdict
import pandas as pd
import scanpy as sc
import numpy as np
import matplotlib.pyplot as plt
from glmpca import glmpca
from itertools import combinations
import torch
import gaston
import seaborn as sns
import math

In [7]:
from gaston import neural_net,cluster_plotting, dp_related, segmented_fit
from gaston import binning_and_plotting, isodepth_scaling, run_slurm_scripts
from gaston import spatial_gene_classification, plot_cell_types, filter_genes, process_NN_output

In [8]:
df = pd.read_csv('lrpairs.csv') #for ligand receptor analysis

df = df.loc[:, ~df.columns.str.contains('^Unnamed')]


# Filtering the DataFrame
mouse_df = df[df['species'] == 'Mouse']

# Display the filtered data
print(mouse_df)

       ligand receptor species
3398     Wnt3     Fzd6   Mouse
3399     Wnt3     Egfr   Mouse
3400     Wnt3     Fzd2   Mouse
3401     Wnt3     Lrp5   Mouse
3402     Wnt3     Fzd1   Mouse
...       ...      ...     ...
5422     Nrg4    Erbb4   Mouse
5423  Adamts3    Ccbe1   Mouse
5424     Bmp6   Acvr2b   Mouse
5425    Pdgfd   Pdgfra   Mouse
5426     Apoe    Vldlr   Mouse

[2029 rows x 3 columns]


In [13]:
CDATADIR = "./"
# counts_mat=np.load(CDATADIR + 'cerebellum_data/cerebellum_counts_mat.npy') # N x G UMI count array
coords_mat=np.load(CDATADIR + 'cerebellum_data/cerebellum_coords_mat.npy') # N x 2 spatial coordinate matrix
gene_labels=np.load(CDATADIR + 'cerebellum_data/cerebellum_gene_labels.npy', allow_pickle=True) # list of names for G genes
# print(counts_mat.shape)

In [16]:
A=np.load(CDATADIR + 'cerebellum_data/F_glmpca_penalty_10_rep1.npy') # GLM-PCA results used in manuscript
S=coords_mat

# z-score normalize S and A
S_torch, A_torch = neural_net.load_rescale_input_data(S,A)

In [17]:
# gaston_model, A, S = gd.gaston_model('tutorial_outputs', S_torch, A_torch)
gaston_model, A, S= process_NN_output.process_files(CDATADIR + 'cerebellum_data/reproduce_cerebellum')


best model: ./cerebellum_data/reproduce_cerebellum/seed19


In [18]:
cell_type_mod_df = gd.create_cell_type_df(CDATADIR + 'cerebellum_data/cell_type_df.csv')

In [23]:
flip = False #True for inverted isodepth
gaston_isodepth, gaston_labels = gd.calculate_isodepth(gaston_model, A, S, coords_mat, num_layers=4,
    scale_factor=64/100, flip=flip, adjust = True)

In [24]:
import warnings
warnings.filterwarnings("ignore")
ct_list= ['Oligodendrocytes', 'Granule', 'Bergmann', 'Purkinje', 'MLI1', 'MLI2']
cell_type_df=pd.read_csv(CDATADIR + 'cerebellum_data/cell_type_df.csv', index_col=0)
pw_fit_dict, binning_output = gd.binning_output(counts_mat, gene_labels, gaston_labels, gaston_isodepth, cell_type_df, ct_list, num_bins_per_domain=[20,40,20,20])

NameError: name 'counts_mat' is not defined

In [ ]:
#CREATE DATASETS

In [21]:
#Iterate over all layers
for l in range(4):
    gaston_isodepth_l=gaston_isodepth[(gaston_labels == l)]
    print((min(gaston_isodepth_l), max(gaston_isodepth_l)))
    coords_mat_l = coords_mat[(gaston_labels == l), :]
    counts_mat_l=counts_mat[(gaston_labels == l), :]
    df = pd.DataFrame(counts_mat_l, columns=gene_labels)
    adata=sc.AnnData(df)
    adata.obsm["coords"] = coords_mat_l
    dag_adjacency_matrix = gd.construct_dag(adata, gaston_isodepth_l, coords_mat_l, n_neighbors=7, plot=False)
    gd.plot_spatial_dag(dag_adjacency_matrix, coords_mat_l, gaston_isodepth_l, legend = True)

    adata_tf = adata.copy()
    adata_lr = adata.copy()
    gd.create_tf_targets(adata_tf, pw_fit_dict, binning_output, q=0.9, file = 'transcription_factors.txt')
    gd.create_lr_targets(adata_lr, pw_fit_dict, binning_output, q=0.9)


(0.0, 95.02807)


NameError: name 'counts_mat' is not defined

In [22]:
#Iterate over boundaries

# boundaries = [95.02807, 180.52618, 370.18195]
if flip:
    boundaries = [180.79575, 370.451, 455.95197] #boundaries between inverted isodepth layers
for l in range(3):
    bound = boundaries[l]
    lower = bound - 50
    upper = bound + 50
    gaston_isodepth_l=gaston_isodepth[(gaston_isodepth >= lower) & (gaston_isodepth <  upper)]

    coords_mat_l = coords_mat[(gaston_isodepth >= lower) & (gaston_isodepth <= upper), :]
    counts_mat_l=counts_mat[(gaston_isodepth >= lower) & (gaston_isodepth <= upper), :]
    df = pd.DataFrame(counts_mat_l, columns=gene_labels)
    adata.obsm["coords"] = coords_mat_l
    dag_adjacency_matrix = gd.construct_dag(adata, gaston_isodepth_l, coords_mat_l, n_neighbors=7, plot=False)
    gd.plot_spatial_dag(dag_adjacency_matrix, coords_mat_l, gaston_isodepth_l, legend = True)
    
    adata_tf = adata.copy()
    adata_lr = adata.copy()
    
    gd.create_tf_targets(adata_tf, pw_fit_dict, binning_output, q=0.9, file = 'transcription_factors.txt')
    gd.create_lr_targets(adata_lr, pw_fit_dict, binning_output, q=0.9)


NameError: name 'boundaries' is not defined